<a href="https://colab.research.google.com/github/Daniel-chagas/NewRepo/blob/master/Deteccao_Conteudo_Inapropriado_Letras_Musicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# Imports essenciais para o projeto

# pandas para manipulação de dados em tabelas
import pandas as pd

# numpy para operações numéricas
import numpy as np

# re para expressões regulares (limpeza de texto)
import re

# matplotlib.pyplot para criar gráficos
import matplotlib.pyplot as plt

# seaborn para gráficos estatísticos mais elaborados
import seaborn as sns

# Tokenizer e pad_sequences do Keras para pré-processamento de texto
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Sequential e camadas do Keras para criar o modelo de rede neural
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense


In [14]:
# Ler o arquivo CSV com todas as músicas e suas informações
df = pd.read_csv('/content/all_songs_data.csv')

# Mostrar as primeiras linhas do DataFrame para verificar se os dados foram carregados corretamente
df.head()


,Album,Album URL,Artist,Featured Artists,Lyrics,Media,Rank,Release Date,Song Title,Song URL,Writers,Year
0,Battle of New Orleans,https://genius.com/albums/Johnny-horton/Battle...,Johnny Horton,[],[Verse 1] In 1814 we took a little trip Along ...,[{'native_uri': 'spotify:track:0dwpdcQkeZqpuoA...,1,1959-04-01,The Battle Of New Orleans,https://genius.com/Johnny-horton-the-battle-of...,"[{'api_path': '/artists/561913', 'header_image...",1959.0
1,That’s All,https://genius.com/albums/Bobby-darin/That-s-all,Bobby Darin,[],"Oh the shark, babe Has such teeth, dear And he...",[{'native_uri': 'spotify:track:3E5ndyOfO6vFDEI...,2,NaN,Mack The Knife,https://genius.com/Bobby-darin-mack-the-knife-...,"[{'api_path': '/artists/218851', 'header_image...",1959.0
2,“Mr Personality’s” 15 Big Hits,https://genius.com/albums/Lloyd-price/Mr-perso...,Lloyd Price,[],Over and over I tried to prove my love to you ...,"[{'provider': 'youtube', 'start': 0, 'type': '...",3,NaN,Personality,https://genius.com/Lloyd-price-personality-lyrics,"[{'api_path': '/artists/355804', 'header_image...",1959.0
3,The Greatest Hits Of Frankie Avalon,https://genius.com/albums/Frankie-avalon/The-g...,Frankie Avalon,[],"Hey, Venus! Oh, Venus! Venus, if you will Ple...",[],4,NaN,Venus,https://genius.com/Frankie-avalon-venus-lyrics,"[{'api_path': '/artists/1113175', 'header_imag...",1959.0
4,Paul Anka Sings His Big 15,https://genius.com/albums/Paul-anka/Paul-anka-...,Paul Anka,[],I'm just a lonely boy Lonely and blue I'm all ...,[],5,NaN,Lonely Boy,https://genius.com/Paul-anka-lonely-boy-lyrics,[],1959.0


In [15]:
# Mostrar os nomes de todas as colunas disponíveis no DataFrame
print("Colunas disponíveis:", df.columns.tolist())

# Verificar quantos valores nulos existem em cada coluna
print("\nValores nulos por coluna:")
print(df.isnull().sum())

# Mostrar quantas músicas existem no dataset
print(f"\nTotal de músicas: {len(df)}")


Colunas disponíveis: ['Album', 'Album URL', 'Artist', 'Featured Artists', 'Lyrics', 'Media', 'Rank', 'Release Date', 'Song Title', 'Song URL', 'Writers', 'Year']

Valores nulos por coluna:
Album                464
Album URL            464
Artist                 0
Featured Artists     116
Lyrics               116
Media                116
Rank                   0
Release Date        1937
Song Title             0
Song URL             116
Writers              116
Year                   0
dtype: int64

Total de músicas: 6500


In [16]:
# Selecionar apenas as colunas que são úteis para o projeto
df_limpo = df[['Song Title', 'Artist', 'Year', 'Lyrics']].copy()

# Remover linhas onde a coluna 'Lyrics' está vazia ou nula
df_limpo = df_limpo.dropna(subset=['Lyrics'])

# Mostrar quantas músicas ficaram após remover letras nulas e visualizar as primeiras linhas
print(f"Músicas com letras válidas: {len(df_limpo)}")
df_limpo.head()


Músicas com letras válidas: 6384


,Song Title,Artist,Year,Lyrics
0,The Battle Of New Orleans,Johnny Horton,1959.0,[Verse 1] In 1814 we took a little trip Along ...
1,Mack The Knife,Bobby Darin,1959.0,"Oh the shark, babe Has such teeth, dear And he..."
2,Personality,Lloyd Price,1959.0,Over and over I tried to prove my love to you ...
3,Venus,Frankie Avalon,1959.0,"Hey, Venus! Oh, Venus! Venus, if you will Ple..."
4,Lonely Boy,Paul Anka,1959.0,I'm just a lonely boy Lonely and blue I'm all ...


In [17]:
# Carregar a base de dados completa já limpa com letras válidas
df = pd.read_csv("/content/all_songs_data.csv")

# Selecionar apenas as colunas importantes e remover linhas sem letras
df_limpo = df[['Song Title', 'Artist', 'Year', 'Lyrics']].dropna(subset=['Lyrics'])

# Criar uma amostra estratificada por ano: seleciona 1 música por ano se possível
amostra_rotulo = df_limpo.groupby('Year').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)

# Verificar se a amostra tem menos de 30 músicas; se tiver, completar com amostras extras
if len(amostra_rotulo) < 30:
    faltam = 30 - len(amostra_rotulo)
    extras = df_limpo.sample(faltam, random_state=42)
    amostra_rotulo = pd.concat([amostra_rotulo, extras], ignore_index=True)

# Adicionar uma coluna vazia chamada 'label' para o rótulo manual
amostra_rotulo['label'] = ''

# Salvar essa amostra em um novo CSV para que o rótulo possa ser preenchido manualmente
caminho_csv = "/content/rotulo_manual_musicas.csv"
amostra_rotulo.to_csv(caminho_csv, index=False)

# Mostrar o caminho do arquivo gerado
caminho_csv


<ipython-input-17-1305321434>:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  amostra_rotulo = df_limpo.groupby('Year').apply(lambda x: x.sample(1, random_state=42)).reset_index(drop=True)


'/content/rotulo_manual_musicas.csv'

In [24]:
# Carregar o arquivo CSV que foi corrigido e agora contém os rótulos preenchidos manualmente
rotulado_corrigido = pd.read_csv("/content/rotulo_manual_musicas_com_label - rotulo_manual_musicas_com_label.csv.csv")

# Visualizar as primeiras linhas para conferir se os dados estão corretos
rotulado_corrigido.head()


,Song Title,Artist,Year,Lyrics,label
0,Don't You Know,Della Reese,1959.0,Don't you know I have fallen in love with you ...,0.4
1,It's Time To Cry,Paul Anka,1960.0,"When somebody leaves you, that's the time to c...",1.0
2,Bless You,Tony Orlando,1961.0,Every time I faced the world I just had to cry...,0.4
3,The Man Who Shot Liberty Valance,Gene Pitney,1962.0,When Liberty Valance rode to town The women fo...,0.6
4,I Wanna Be Around,Tony Bennett,1963.0,"I wanna be around, to pick up the pieces - Whe...",1.0


In [ ]:
!pip install tensorflow


In [25]:
# Função para fazer uma limpeza básica do texto das letras
def limpar_texto(texto):
    texto = str(texto).lower()  # garantir que o texto seja string e converter para minúsculo
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)  # remover tudo que não for letra ou espaço
    return texto

# Aplicar a função de limpeza em todas as letras e criar uma nova coluna com o texto limpo
rotulado_corrigido['letra_limpa'] = rotulado_corrigido['Lyrics'].apply(limpar_texto)


In [26]:
# Criar o tokenizador para converter texto em números
# Limitar o vocabulário aos 5000 termos mais comuns e definir um token para palavras desconhecidas
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")

# Ajustar o tokenizador com base nos textos limpos
tokenizer.fit_on_texts(rotulado_corrigido['letra_limpa'])

# Transformar cada texto limpo em uma sequência de números (tokens)
sequences = tokenizer.texts_to_sequences(rotulado_corrigido['letra_limpa'])


In [27]:
# Definir o comprimento máximo das sequências de tokens
max_len = 200

# Aplicar o padding para garantir que todas as sequências tenham o mesmo tamanho
# 'post' adiciona ou corta no final da sequência
X = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')


In [28]:
# Converter a coluna 'label' para tipo float e transformar em um array NumPy
# Isso garante que os rótulos estejam no formato certo para treinar o modelo
y = rotulado_corrigido['label'].astype(float).values


In [29]:
# Importar a função para dividir os dados em treino e teste
from sklearn.model_selection import train_test_split

# Dividir os dados em 80% para treino e 20% para teste
# O random_state garante que a divisão seja a mesma toda vez que rodar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [30]:
# Criar um modelo sequencial simples

modelo = Sequential()

# Camada de embedding para transformar tokens em vetores de tamanho 32
modelo.add(Embedding(input_dim=5000, output_dim=32, input_length=max_len))

# Camada de pooling para gerar um vetor fixo fazendo média sobre a sequência
modelo.add(GlobalAveragePooling1D())

# Camada densa intermediária com 16 neurônios e função de ativação ReLU
modelo.add(Dense(16, activation='relu'))

# Camada de saída com 1 neurônio e ativação sigmoide para prever valor entre 0 e 1
modelo.add(Dense(1, activation='sigmoid'))

# Compilar o modelo com otimizador Adam e erro quadrático médio para regressão
modelo.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [31]:
# Treinar o modelo com os dados de treino
# Usar 30 épocas, batch size de 4 e validar com os dados de teste
# verbose=1 para mostrar o progresso durante o treino
modelo.fit(X_train, y_train, epochs=30, batch_size=4, validation_data=(X_test, y_test), verbose=1)


Epoch 1/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - loss: 0.1283 - mae: 0.3083 - val_loss: 0.1083 - val_mae: 0.2855
Epoch 2/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1431 - mae: 0.3393 - val_loss: 0.1082 - val_mae: 0.2862
Epoch 3/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1046 - mae: 0.2820 - val_loss: 0.1073 - val_mae: 0.2862
Epoch 4/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1499 - mae: 0.3568 - val_loss: 0.1070 - val_mae: 0.2863
Epoch 5/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1110 - mae: 0.2897 - val_loss: 0.1075 - val_mae: 0.2880
Epoch 6/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1242 - mae: 0.3184 - val_loss: 0.1074 - val_mae: 0.2880
Epoch 7/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1118 - mae: 0.2959 - val_loss: 0.1083 - val_mae: 0.2895
Epoch 8/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.1031 - mae: 0.2871 - val_loss: 0.1096 - val_mae: 0.2912
Epoch 9/30
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0821 

In [32]:
# Avaliar o desempenho do modelo usando os dados de teste
loss, mae = modelo.evaluate(X_test, y_test)

# Mostrar o erro médio absoluto (MAE) obtido na avaliação
print("MAE (Erro médio absoluto):", mae)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.1719 - mae: 0.3682
MAE (Erro médio absoluto): 0.3681933283805847


In [33]:
# Recarregar o dataset completo com todas as músicas
df_completo = pd.read_csv("/content/all_songs_data.csv")

# Selecionar apenas as colunas relevantes e remover linhas sem letras
df_limpo = df_completo[['Song Title', 'Artist', 'Year', 'Lyrics']].dropna(subset=['Lyrics']).copy()


In [34]:
# Função para fazer a limpeza básica do texto das letras
def limpar_texto(texto):
    texto = texto.lower()  # converter para minúsculas
    texto = re.sub(r'[^a-zA-Z\s]', '', texto)  # remover tudo que não for letra ou espaço
    return texto

# Aplicar a função de limpeza em todas as letras e criar uma nova coluna com o texto limpo
df_limpo['letra_limpa'] = df_limpo['Lyrics'].apply(limpar_texto)


In [35]:
# Tokenizar os textos limpos usando o mesmo tokenizador já treinado anteriormente
seqs_completas = tokenizer.texts_to_sequences(df_limpo['letra_limpa'])

# Aplicar padding para garantir que todas as sequências tenham o mesmo comprimento
X_completo = pad_sequences(seqs_completas, maxlen=200, padding='post', truncating='post')


In [36]:
# Usar o modelo treinado para prever a pontuação de toxicidade de todas as músicas
# O resultado é salvo em uma nova coluna chamada 'pontuacao_prevista'
df_limpo['pontuacao_prevista'] = modelo.predict(X_completo, verbose=0)


In [37]:
# Garantir que a coluna 'pontuacao_prevista' esteja no formato float com ponto decimal
df_limpo['pontuacao_prevista'] = df_limpo['pontuacao_prevista'].astype(float)


In [38]:
# Exibir estatísticas básicas da coluna 'pontuacao_prevista'
# Isso mostra mínimo, máximo, média, quartis, etc.
df_limpo['pontuacao_prevista'].describe()


,pontuacao_prevista
count,6384.000000
mean,0.486975
std,0.165601
min,0.017164
25%,0.373977
50%,0.480264
75%,0.593152
max,0.930844


In [39]:
# Arredondar os valores da coluna 'pontuacao_prevista' para 1 casa decimal
df_limpo['pontuacao_prevista'] = df_limpo['pontuacao_prevista'].round(1)

# Verificar a distribuição dos valores após o arredondamento
# value_counts(normalize=True) mostra as proporções relativas
df_limpo['pontuacao_prevista'].value_counts(normalize=True).sort_index()


,proportion
pontuacao_prevista,
0.0,0.001723
0.1,0.011591
0.2,0.058427
0.3,0.132049
0.4,0.222274
0.5,0.238565
0.6,0.174029
0.7,0.090226
0.8,0.050125


In [40]:
# Salvar o DataFrame com as pontuações previstas em um novo arquivo CSV
# O parâmetro index=False garante que o índice não seja salvo como coluna
df_limpo.to_csv("/content/musicas_com_pontuacao_prevista.csv", index=False)


In [41]:
# Mostrar uma amostra dos resultados ordenada pela pontuação prevista em ordem decrescente
# Exibir apenas as colunas de título da música, artista, ano e pontuação prevista
# Mostrar as 10 músicas com maior pontuação prevista
df_limpo[['Song Title', 'Artist', 'Year', 'pontuacao_prevista']].sort_values(by='pontuacao_prevista', ascending=False).head(10)


,Song Title,Artist,Year,pontuacao_prevista
21,The Happy Organ,"Dave ""Baby"" Cortez",1959.0,0.9
6482,Wait for U,Future featuring Drake and Tems,2023.0,0.9
6496,Meltdown,Travis Scott featuring Drake,2023.0,0.9
23,Sorry (I Ran All The Way Home),Impalas,1959.0,0.9
5952,Heaven,Kane Brown,2018.0,0.9
5963,Sorry Not Sorry,Demi Lovato,2018.0,0.9
10,Sleep Walk,Santo and Johnny,1959.0,0.9
6480,What Was I Made For?,Billie Eilish,2023.0,0.9
1160,25 Or 6 To 4,Chicago,1970.0,0.9
35,Teen Beat,Sandy Nelson,1959.0,0.9


from matplotlib import pyplot as plt
_df_0['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_1.sort_values('Year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['pontuacao_prevista']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'pontuacao_prevista'}, axis=1)
              .sort_values('pontuacao_prevista', ascending=True))
  xs = counted['pontuacao_prevista']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_2.sort_values('pontuacao_prevista', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('pontuacao_prevista')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_3['Year'].plot(kind='line', figsize=(8, 4), title='Year')
plt.gca().spines[['top', 'right']].set_visible(False)